PART ONE:

Import libraries:

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import seaborn as sns

#import beautiful libraries
from bs4 import BeautifulSoup

import urllib

print('Libraries imported.')

Libraries imported.


Import data into dataframe from web page:

In [4]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Eliminating rows of data with no assigned borough:

In [5]:
toronto_df = df[~df.Borough.str.contains("Not assigned")]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Renaming columns for simplicity:

In [6]:
toronto_df.rename(columns = {'Postal Code': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace = True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Combining Neighborhoods with the same Postal Code:

In [7]:
df.groupby([toronto_df.PostalCode]).apply(lambda x: "{%s}" % ', '.join(x.Neighbourhood))
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Shape of dataframe:

In [8]:
toronto_df.shape

(103, 3)

PART TWO:

Import libraries:

In [9]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

Read geo file into dataframe:

In [10]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
                     
print('Data read to into pandas dataframe!')

Data read to into pandas dataframe!


Rename Columns for simplicity:

In [11]:
geo_df.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

Merge data frames:

In [12]:
ll_df = pd.merge(left = geo_df, right = toronto_df, right_on = 'PostalCode', left_on = 'PostalCode')
ll_df.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [13]:
for data in ll_df:
    PostalCode = ll_df['PostalCode']
    borough = neighborhood_name = ll_df['Borough'] 
    neighborhood_name = ll_df['Neighborhood']      
    neighborhood_lat = ll_df['Latitude']
    neighborhood_lon = ll_df['Longitude']
    
    neighborhoods = ll_df.append({'PostalCode': PostalCode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


Reorder columns:

In [14]:
final_df = neighborhoods[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
final_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


PART THREE:

In [30]:

!pip install folium
import folium

Calculate Toronto city geographical coordinates:

In [31]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

ValueError: Location should consist of two numerical values, but 0      43.806686
1      43.784535
2      43.763573
3      43.770992
4      43.773136
5      43.744734
6      43.727929
7      43.711112
8      43.716316
9      43.692657
10     43.757410
11     43.750072
12     43.794200
13     43.781638
14     43.815252
15     43.799525
16     43.836125
17     43.803762
18     43.778517
19     43.786947
20     43.757490
21     43.789053
22     43.770120
23     43.752758
24     43.782736
25     43.753259
26     43.745906
27     43.725900
28     43.754328
29     43.767980
         ...    
73     43.693781
74     43.689026
75     43.669542
76     43.669005
77     43.647927
78     43.636847
79     43.713756
80     43.691116
81     43.673185
82     43.661608
83     43.648960
84     43.651571
85     43.662301
86     43.636966
87     43.662744
88     43.605647
89     43.602414
90     43.653654
91     43.636258
92     43.628841
93     43.667856
94     43.650943
95     43.643515
96     43.756303


Defining Four Square credentials:

In [175]:
CLIENT_ID = '1CNV51WDL0HVJF1YLVIFT0NI0AK1HBDC2S2T0D4M0UCVINKW' # your Foursquare ID
CLIENT_SECRET = 'KRXO4PJIWLRWWCZY5JVSRHOJXHHIFRZYEGYOOECCUFEUDSK0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1CNV51WDL0HVJF1YLVIFT0NI0AK1HBDC2S2T0D4M0UCVINKW
CLIENT_SECRET:KRXO4PJIWLRWWCZY5JVSRHOJXHHIFRZYEGYOOECCUFEUDSK0
